In [137]:
from gsheets import Sheets

In [13]:
sheets = Sheets.from_files('~/documents/developer/go_bot/client_secrets.json', '~/documents/developer/go_bot/storage.json')


Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=815260405548-l9lluqtgamfoe0s284cvmtp0g76ovamj.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fspreadsheets.readonly+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.readonly&access_type=offline&response_type=code

If your browser is on a different machine then exit and re-run this
application with the command-line parameter

  --noauth_local_webserver

Authentication successful.


In [15]:
url = 'https://docs.Sheets.com/spreadsheets/d/1XSV89pZD1gj8DOTIu9asL75Dz4n4Rl-h7rciadjvwF8'

In [18]:
s = sheets.get(url)

In [22]:
frame = s.sheets[0].to_frame()

In [27]:
frame

,Smart IGN,PFID,Multi Account PFID,GO Rating,30+ Rating
0,TTP.Guramichi15,79ED45AD17C8483B,79ED45AD17C8483B,1946.318083,2698.724418
1,MyVRUser12,764DD262873FF5FD,764DD262873FF5FD,1919.545739,2690.070835
2,Gabe_Watts.NB,897EC5B9C24E425E,897EC5B9C24E425E,1882.838221,2613.429481
3,Scanter,9CC35B4B3A8E41D1,9CC35B4B3A8E41D1,1879.689239,2767.670788
4,Legends_1,8F5542CC5FA8A54F,8F5542CC5FA8A54F,1848.905708,2637.606884
...,...,...,...,...,...
2925,Jen_Beth,CA392AAFB8701AF3,CA392AAFB8701AF3,147.572816,36.026764
2926,2Souls,D38B995C8848C0A9,D38B995C8848C0A9,146.241830,13.899732
2927,TaterTot2.0_TSK,6E499AB0A4530804,6E499AB0A4530804,142.293395,53.108748
2928,TRO_dorminjones,BE05DA3CD5BFB3F8,BE05DA3CD5BFB3F8,82.749767,18.723590


In [163]:
cd /Users/stube/Documents/developer/GoBot/

/Users/stube/Documents/developer/GoBot


In [164]:
pwd

'/Users/stube/Documents/developer/GoBot'

In [165]:
import sqlite3

In [211]:
con = sqlite3.connect("gobot.db")

def attrs_factory(cursor, row):
    fields = [column[0] for column in cursor.description]
    Row = attrs.make_class("Row", fields)
    r = Row(*row)
    # print(f"attrs_factory row {r}")
    return r

con.row_factory = attrs_factory

cur = con.cursor()

In [212]:
res = cur.execute("SELECT name FROM sqlite_master")
tables = [row.name for row in res.fetchall()]
print("teams" in tables)
tables

True


['teams',
 'sqlite_autoindex_teams_1',
 'players',
 'rosters',
 'signups',
 'ratings',
 'sqlite_autoindex_ratings_1']

In [213]:
    cur.execute("CREATE TABLE teams(team_id INTEGER PRIMARY KEY, team_name UNIQUE, player_count)")
    cur.execute("CREATE TABLE players(discord_id, discord_name, pfid, ign)")
    cur.execute("CREATE TABLE rosters(team_id, discord_id)")
    cur.execute("CREATE TABLE signups(date, team_id)")
    cur.execute("CREATE TABLE ratings(pfid UNIQUE, ign, go_rating)")


OperationalError: table teams already exists

In [185]:
cur.execute('INSERT INTO players VALUES (895726168140038185, "go_shovvtimex", 0, "GO Showtime")')
cur.execute('INSERT INTO players VALUES (497806556008546304, "go_maverick", 0, "GO Mav")')
cur.execute('INSERT INTO players VALUES (904075134938267707, "go_smartazz", 0, "GO SmartAzz")')
con.commit()

In [186]:
con.close()

In [214]:
cur.execute("SELECT * FROM players")
cur.fetchall()

[Row(discord_id=408731638223208448, discord_name='GO_STOOOBE', pfid=0, ign='GO_STOOOBE'),
 Row(discord_id=408731638223208448, discord_name='GO_STOOOBE', pfid=0, ign='GO_STOOOBE'),
 Row(discord_id=408731638223208448, discord_name='GO_STOOOBE', pfid=0, ign='GO_STOOOBE'),
 Row(discord_id=161995862153625600, discord_name='VIPxRyan2dayy', pfid=0, ign='VIPxRyan2dayy'),
 Row(discord_id=161995862153625600, discord_name='VIPxRyan2dayy', pfid=0, ign='VIPxRyan2dayy'),
 Row(discord_id=161995862153625600, discord_name='VIPxRyan2dayy', pfid=0, ign='VIPxRyan2dayy'),
 Row(discord_id=795494347876597770, discord_name='VIPxMuggy', pfid=0, ign='VIPxMuggy'),
 Row(discord_id=795494347876597770, discord_name='VIPxMuggy', pfid=0, ign='VIPxMuggy'),
 Row(discord_id=795494347876597770, discord_name='VIPxMuggy', pfid=0, ign='VIPxMuggy'),
 Row(discord_id=826046643626835988, discord_name='MEME', pfid=0, ign='MEME'),
 Row(discord_id=826046643626835988, discord_name='MEME', pfid=0, ign='MEME'),
 Row(discord_id=826046

In [210]:
cur.execute("SELECT t.team_id, t.team_name, p.discord_name, p.ign "+
            "FROM teams as t, rosters as r, players as p "+
            "WHERE r.team_id=t.team_id and r.discord_id=p.discord_id")
cur.fetchall()

[Row(team_id=1, team_name='tn1', discord_name='GO_STOOOBE', ign='STOOBE'),
 Row(team_id=2, team_name='tn2', discord_name='go_shovvtimex', ign='GO Showtime'),
 Row(team_id=3, team_name='tn3', discord_name='go_maverick', ign='GO Mav'),
 Row(team_id=3, team_name='tn3', discord_name='go_smartazz', ign='GO SmartAzz')]

In [205]:
cur.execute("SELECT s.date, r.team_id, r.discord_id "+
            "FROM signups as s, rosters as r "+
            "WHERE s.team_id=r.team_id and discord_id=895726168140038185")
cur.fetchall()

[Row(date='2024-01-20', team_id=2, discord_id=895726168140038185)]

In [206]:
import attrs

In [207]:
Row = attrs.make_class("Row", ["a", "b", "d", "c"])
Row(1,2,3,4)

Row(a=1, b=2, d=3, c=4)

In [208]:
cur.execute("SELECT team_id FROM rosters WHERE discord_id=?", (408731638223208448,))
cur.fetchall()

[Row(team_id=1)]